In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Optimizing 5D Parallelism for Training a Financial Domain LLM
## Implementation Notebook

In this notebook, you will work through the core technical challenge facing Meridian AI: configuring a 5D parallelism strategy (Data, Tensor, Pipeline, Sequence, and Expert Parallelism) to train a 72B-parameter Mixture-of-Experts language model on 1,024 H100 GPUs.

You will work at a reduced scale — a 1.3B parameter, 8-expert MoE model on a simulated 8-GPU setup — but every parallelism dimension and tradeoff is preserved. By the end, you will have built a parallelism configuration optimizer that generalizes to any model size and cluster.

**Prerequisites:** PyTorch 2.x, basic understanding of distributed computing, familiarity with Transformer architecture.

---

## 3.1 Data Acquisition Strategy

We use two datasets for this case study:

1. **SEC-EDGAR Filings** (sampled) — A subset of 10-K annual filings from publicly traded companies, representing the long-document workload that MeridianFin must handle.
2. **FinanceBench** — A curated QA dataset grounded in SEC filings, used for post-training evaluation.

For the parallelism experiments, we generate synthetic token sequences that match the statistical profile of financial text (sequence length distribution, vocabulary usage). This lets us focus on the systems engineering aspects without requiring the full 4T-token corpus.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/5d-parallelism-from-scratch/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
# Setup and imports
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.nn.functional as F
from torch.distributed import ProcessGroup
import math
import time
import os
from dataclasses import dataclass
from typing import Optional, Tuple, List, Dict
import json

# Check GPU availability
if torch.cuda.is_available():
    print(f"CUDA available: {torch.cuda.device_count()} GPU(s)")
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"  GPU {i}: {props.name}, {props.total_memory / 1e9:.1f} GB")
else:
    print("No CUDA GPUs available. This notebook requires GPU(s).")
    print("In Colab: Runtime -> Change runtime type -> T4 GPU")

In [ ]:
# Configuration dataclass for the model and hardware
@dataclass
class ModelConfig:
    """Configuration for the MoE Transformer model."""
    vocab_size: int = 32000
    hidden_dim: int = 2048          # d_model
    num_layers: int = 24            # Total transformer layers
    num_heads: int = 16             # Attention heads
    head_dim: int = 128             # Per-head dimension
    ffn_dim: int = 5504             # MLP intermediate dimension (non-expert layers)
    num_experts: int = 8            # Number of MoE experts
    top_k: int = 2                  # Top-K expert routing
    expert_ffn_dim: int = 2048      # Per-expert MLP intermediate dimension
    max_seq_len: int = 4096         # Maximum sequence length (reduced scale)
    dropout: float = 0.0

    @property
    def total_params(self) -> int:
        """Estimate total parameter count."""
        # Embedding
        embed = self.vocab_size * self.hidden_dim
        # Per non-MoE layer: attention + MLP
        attn_per_layer = 4 * self.hidden_dim * self.hidden_dim  # Q, K, V, O projections
        mlp_per_layer = 2 * self.hidden_dim * self.ffn_dim      # up + down projections
        # Per MoE layer: attention + N experts + router
        expert_params = self.num_experts * 2 * self.hidden_dim * self.expert_ffn_dim
        router_params = self.hidden_dim * self.num_experts
        # Assume half the layers are MoE, half are dense (simplified)
        n_moe_layers = self.num_layers // 2
        n_dense_layers = self.num_layers - n_moe_layers
        total = (embed
                + n_dense_layers * (attn_per_layer + mlp_per_layer)
                + n_moe_layers * (attn_per_layer + expert_params + router_params)
                + embed)  # output projection (tied weights typically, but count for memory)
        return total

    @property
    def active_params(self) -> int:
        """Estimate active parameters per token (with top-K routing)."""
        embed = self.vocab_size * self.hidden_dim
        attn_per_layer = 4 * self.hidden_dim * self.hidden_dim
        mlp_per_layer = 2 * self.hidden_dim * self.ffn_dim
        active_expert_params = self.top_k * 2 * self.hidden_dim * self.expert_ffn_dim
        router_params = self.hidden_dim * self.num_experts
        n_moe_layers = self.num_layers // 2
        n_dense_layers = self.num_layers - n_moe_layers
        total = (embed
                + n_dense_layers * (attn_per_layer + mlp_per_layer)
                + n_moe_layers * (attn_per_layer + active_expert_params + router_params)
                + embed)
        return total


@dataclass
class HardwareConfig:
    """Configuration for the GPU cluster."""
    num_gpus: int = 8               # Total GPUs (reduced scale)
    gpus_per_node: int = 8          # GPUs per node
    gpu_memory_gb: float = 80.0     # Memory per GPU in GB (simulated)
    nvlink_bw_gbps: float = 900.0   # NVLink bandwidth (GB/s)
    ib_bw_gbps: float = 50.0        # InfiniBand bandwidth per link (GB/s)
    num_ib_links: int = 8           # IB links per node
    peak_tflops: float = 989.0      # BF16 peak TFLOPS per GPU (H100 SXM)

    @property
    def num_nodes(self) -> int:
        return self.num_gpus // self.gpus_per_node


@dataclass
class ParallelismConfig:
    """5D parallelism configuration."""
    dp: int = 1     # Data Parallelism degree
    tp: int = 1     # Tensor Parallelism degree
    pp: int = 1     # Pipeline Parallelism degree
    sp: int = 1     # Sequence/Context Parallelism degree
    ep: int = 1     # Expert Parallelism degree
    zero_stage: int = 0         # ZeRO optimization stage (0, 1, 2, 3)
    num_microbatches: int = 1   # Micro-batches for pipeline parallelism
    activation_checkpointing: bool = False

    def validate(self, hw: HardwareConfig, model: ModelConfig) -> List[str]:
        """Validate the configuration against hardware and model constraints."""
        errors = []
        # GPU count constraint
        if self.dp * self.tp * self.pp * self.ep != hw.num_gpus:
            errors.append(
                f"DP({self.dp}) x TP({self.tp}) x PP({self.pp}) x EP({self.ep}) = "
                f"{self.dp * self.tp * self.pp * self.ep} != {hw.num_gpus} GPUs"
            )
        # TP within node
        if self.tp > hw.gpus_per_node:
            errors.append(f"TP({self.tp}) > GPUs per node({hw.gpus_per_node})")
        # Layer divisibility
        if model.num_layers % self.pp != 0:
            errors.append(f"Layers({model.num_layers}) not divisible by PP({self.pp})")
        # Expert divisibility
        if model.num_experts % self.ep != 0:
            errors.append(f"Experts({model.num_experts}) not divisible by EP({self.ep})")
        # SP must equal TP (Megatron-style)
        if self.sp != self.tp and self.sp != 1:
            errors.append(f"SP({self.sp}) must equal TP({self.tp}) or be 1")
        return errors


# Initialize configurations
model_cfg = ModelConfig()
hw_cfg = HardwareConfig()

print(f"Model: {model_cfg.total_params / 1e9:.2f}B total parameters")
print(f"Model: {model_cfg.active_params / 1e9:.2f}B active parameters per token")
print(f"Hardware: {hw_cfg.num_gpus} GPUs across {hw_cfg.num_nodes} node(s)")

In [ ]:
# Data pipeline: Synthetic financial text sequences for parallelism experiments

def create_synthetic_dataset(
    num_sequences: int,
    seq_length: int,
    vocab_size: int,
    seed: int = 42
) -> torch.Tensor:
    """
    Create synthetic token sequences for parallelism experiments.

    In production, this would be replaced by the tokenized SEC-EDGAR corpus.
    We use synthetic data here to focus on the systems engineering aspects.

    Args:
        num_sequences: Number of sequences to generate
        seq_length: Tokens per sequence
        vocab_size: Vocabulary size
        seed: Random seed for reproducibility

    Returns:
        Tensor of shape (num_sequences, seq_length) with token IDs
    """
    torch.manual_seed(seed)
    # Zipf-like distribution to mimic natural language token frequencies
    probs = 1.0 / torch.arange(1, vocab_size + 1).float()
    probs = probs / probs.sum()
    tokens = torch.multinomial(probs.expand(num_sequences, -1), seq_length, replacement=True)
    return tokens


# TODO: Implement a data-parallel-aware data loader
def create_dp_dataloader(
    dataset: torch.Tensor,
    dp_rank: int,
    dp_world_size: int,
    micro_batch_size: int,
    shuffle: bool = True,
    seed: int = 42
) -> List[torch.Tensor]:
    """
    Shard the dataset across Data Parallel ranks and return micro-batches.

    Each DP rank should see a DISJOINT subset of the data. This is critical:
    if two DP ranks see the same data, we waste compute and introduce bias.

    Args:
        dataset: Full dataset tensor of shape (N, seq_len)
        dp_rank: This rank's position in the DP group (0 to dp_world_size-1)
        dp_world_size: Total number of DP ranks
        micro_batch_size: Number of sequences per micro-batch
        shuffle: Whether to shuffle before sharding
        seed: Random seed for reproducible shuffling

    Returns:
        List of micro-batch tensors, each of shape (micro_batch_size, seq_len)

    Hints:
        1. If shuffle=True, generate a permutation of indices using the seed
        2. Shard the (shuffled) indices: rank i gets indices[i::dp_world_size]
        3. Group the shard into micro-batches of size micro_batch_size
        4. Drop the last incomplete micro-batch if it exists
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement DP-aware data sharding")

In [ ]:
# Verification: Test your data loader
dataset = create_synthetic_dataset(1024, 128, model_cfg.vocab_size)
print(f"Dataset shape: {dataset.shape}")

# Simulate 4-way DP: each rank should get 256 sequences
for rank in range(4):
    batches = create_dp_dataloader(dataset, dp_rank=rank, dp_world_size=4, micro_batch_size=8)
    total_seqs = sum(b.shape[0] for b in batches)
    print(f"  DP rank {rank}: {len(batches)} micro-batches, {total_seqs} sequences total")

# Verify no overlap between ranks
all_indices_r0 = set(range(0, 1024, 4))  # Expected for rank 0 with stride sharding
print(f"\nExpected ~256 sequences per rank, ~32 micro-batches per rank")

**Thought Questions:**
1. Why is it important that DP ranks see disjoint subsets of the data? What would happen to gradient estimates if ranks processed overlapping data?
2. In the production system with 4T tokens, the dataset is too large to fit in memory. How would you modify this data loader to stream from disk?
3. How does the micro-batch size interact with Pipeline Parallelism? (Hint: more micro-batches reduce the pipeline bubble.)

---

## 3.2 Exploratory Data Analysis: Profiling the Workload

Before optimizing the parallelism configuration, we need to understand where the computational and memory bottlenecks are. This section builds a memory model from first principles.

In [ ]:
def estimate_memory_bytes(
    model: ModelConfig,
    par: ParallelismConfig,
    micro_batch_size: int,
    seq_length: int,
    bytes_per_param_training: int = 16  # BF16 weights + grads + FP32 optimizer
) -> Dict[str, float]:
    """
    Estimate per-GPU memory consumption for a given parallelism config.

    This is the analytical memory model from Section 2 of the case study.
    Understanding this model is essential for choosing the right configuration.

    Args:
        model: Model architecture configuration
        par: Parallelism configuration
        micro_batch_size: Micro-batch size per GPU
        seq_length: Sequence length
        bytes_per_param_training: Bytes per parameter for training state (16 for BF16+Adam)

    Returns:
        Dictionary with memory breakdown in bytes:
        - 'model_states': Weights + gradients + optimizer states
        - 'activations': Intermediate activations (per layer, summed)
        - 'attention': Attention score matrices
        - 'communication': Estimated communication buffer overhead
        - 'total': Sum of all components
    """
    # Model states memory
    # With parallelism, parameters are distributed across TP, PP, and EP groups.
    # With ZeRO, the DP dimension further shards the states.
    zero_factor = par.dp if par.zero_stage == 3 else (
        par.dp if par.zero_stage == 2 else (
            par.dp if par.zero_stage == 1 else 1
        )
    )
    # ZeRO-1 only shards optimizer states (12 bytes out of 16)
    # ZeRO-2 shards optimizer states + gradients (14 out of 16)
    # ZeRO-3 shards everything (16 out of 16)
    if par.zero_stage == 0:
        model_bytes_per_param = bytes_per_param_training
    elif par.zero_stage == 1:
        model_bytes_per_param = 4 + 12.0 / par.dp  # weights+grads local, optimizer sharded
    elif par.zero_stage == 2:
        model_bytes_per_param = 2 + 14.0 / par.dp  # weights local, grads+optimizer sharded
    else:  # stage 3
        model_bytes_per_param = bytes_per_param_training / par.dp

    params_per_gpu = model.total_params / (par.tp * par.pp * par.ep)
    model_states = params_per_gpu * model_bytes_per_param

    # Activation memory per layer
    # Each transformer layer stores intermediate activations for the backward pass.
    # Key activations: input to each sublayer, attention scores, MLP intermediates
    layers_per_gpu = model.num_layers // par.pp
    sp_factor = par.sp if par.sp > 1 else 1

    # Simplified activation estimate: ~34 bytes per element per layer
    # (covers attention input/output, MLP input/intermediate/output, residuals, norms)
    act_elements_per_layer = micro_batch_size * (seq_length // sp_factor) * model.hidden_dim
    act_bytes_per_layer = act_elements_per_layer * 34  # empirical constant

    if par.activation_checkpointing:
        # With checkpointing, we only store layer inputs (recompute during backward)
        # Memory is ~2x a single layer's activations regardless of depth
        activations = act_bytes_per_layer * 2
    else:
        activations = act_bytes_per_layer * layers_per_gpu

    # Attention memory (quadratic in sequence length)
    heads_per_gpu = model.num_heads // par.tp
    seq_per_gpu = seq_length // sp_factor
    # Attention scores: (batch, heads, seq, seq) in BF16
    attention = micro_batch_size * heads_per_gpu * seq_per_gpu * seq_length * 2  # 2 bytes for BF16

    # Communication buffers (rough estimate)
    communication = 2 * params_per_gpu * 2 / par.tp  # AllReduce buffers

    total = model_states + activations + attention + communication

    return {
        'model_states': model_states,
        'activations': activations,
        'attention': attention,
        'communication': communication,
        'total': total,
    }

In [ ]:
# Profile memory for the baseline configuration (pure FSDP, no TP/PP/EP)
baseline_par = ParallelismConfig(dp=8, tp=1, pp=1, sp=1, ep=1, zero_stage=3)
baseline_mem = estimate_memory_bytes(model_cfg, baseline_par, micro_batch_size=1, seq_length=4096)

print("=== Baseline Memory Profile (FSDP ZeRO-3, 8 GPUs) ===")
for key, val in baseline_mem.items():
    print(f"  {key:20s}: {val / 1e9:8.2f} GB")

print(f"\n  GPU memory budget: {hw_cfg.gpu_memory_gb:.0f} GB")
print(f"  Fits in memory: {'YES' if baseline_mem['total'] / 1e9 <= hw_cfg.gpu_memory_gb else 'NO'}")

In [ ]:
# TODO: Profile memory for different sequence lengths and identify the breaking point

def find_max_sequence_length(
    model: ModelConfig,
    par: ParallelismConfig,
    hw: HardwareConfig,
    micro_batch_size: int = 1,
    safety_margin_gb: float = 4.0
) -> int:
    """
    Find the maximum sequence length that fits in GPU memory for a given config.

    Binary search over sequence lengths to find the largest one where
    total memory <= gpu_memory - safety_margin.

    Args:
        model: Model configuration
        par: Parallelism configuration
        hw: Hardware configuration
        micro_batch_size: Micro-batch size
        safety_margin_gb: Reserved memory for framework overhead (GB)

    Returns:
        Maximum feasible sequence length (rounded down to nearest 256)

    Hints:
        1. Use binary search between seq_length=256 and seq_length=model.max_seq_len
        2. For each candidate, call estimate_memory_bytes and check against budget
        3. Round down to nearest multiple of 256 for practical alignment
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement binary search for max sequence length")


# Test: Find max seq length for the baseline config
max_seq = find_max_sequence_length(model_cfg, baseline_par, hw_cfg)
print(f"Maximum sequence length for baseline config: {max_seq} tokens")
print(f"Target sequence length: {model_cfg.max_seq_len} tokens")
if max_seq < model_cfg.max_seq_len:
    print("PROBLEM: Baseline cannot handle the target sequence length!")

In [ ]:
# Visualization: Memory breakdown across different sequence lengths
import matplotlib.pyplot as plt

seq_lengths = [512, 1024, 2048, 4096, 8192, 16384]
components = ['model_states', 'activations', 'attention', 'communication']
colors = ['#2196F3', '#4CAF50', '#FF9800', '#9C27B0']

fig, ax = plt.subplots(figsize=(10, 6))
bottoms = [0] * len(seq_lengths)

for comp, color in zip(components, colors):
    values = []
    for sl in seq_lengths:
        mem = estimate_memory_bytes(model_cfg, baseline_par, micro_batch_size=1, seq_length=sl)
        values.append(mem[comp] / 1e9)
    ax.bar(range(len(seq_lengths)), values, bottom=bottoms, label=comp, color=color)
    bottoms = [b + v for b, v in zip(bottoms, values)]

ax.axhline(y=hw_cfg.gpu_memory_gb, color='red', linestyle='--', linewidth=2, label='GPU Memory (80 GB)')
ax.set_xticks(range(len(seq_lengths)))
ax.set_xticklabels([str(sl) for sl in seq_lengths])
ax.set_xlabel('Sequence Length (tokens)')
ax.set_ylabel('Memory (GB)')
ax.set_title('Per-GPU Memory Breakdown vs Sequence Length (Baseline FSDP)')
ax.legend()
plt.tight_layout()
plt.show()

**Thought Questions:**
1. Which memory component grows fastest with sequence length? Why does this matter for long-context training?
2. At what sequence length does attention memory dominate? How does this change with Tensor Parallelism (which splits attention heads)?
3. The memory model uses a constant 34 bytes/element for activations. What are the actual intermediate tensors stored, and why is 34 a reasonable estimate?

---

## 3.3 Baseline Approach: Pure Data Parallelism (FSDP)

We implement a minimal FSDP training loop to establish baseline throughput and identify bottlenecks. Since Colab may have a single GPU, we simulate multi-GPU behavior by running the memory model analytically and implementing the key computation kernels on a single GPU.

In [ ]:
class SimpleTransformerLayer(nn.Module):
    """A simplified Transformer layer for profiling (no MoE, single GPU)."""

    def __init__(self, config: ModelConfig):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.hidden_dim)
        self.attn_qkv = nn.Linear(config.hidden_dim, 3 * config.hidden_dim, bias=False)
        self.attn_out = nn.Linear(config.hidden_dim, config.hidden_dim, bias=False)
        self.norm2 = nn.LayerNorm(config.hidden_dim)
        self.mlp_up = nn.Linear(config.hidden_dim, config.ffn_dim, bias=False)
        self.mlp_down = nn.Linear(config.ffn_dim, config.hidden_dim, bias=False)
        self.num_heads = config.num_heads
        self.head_dim = config.head_dim

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, S, D = x.shape
        # Self-attention
        h = self.norm1(x)
        qkv = self.attn_qkv(h).reshape(B, S, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.unbind(dim=2)
        q = q.transpose(1, 2)  # (B, H, S, D_h)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        attn = F.scaled_dot_product_attention(q, k, v)
        attn = attn.transpose(1, 2).reshape(B, S, D)
        x = x + self.attn_out(attn)
        # MLP
        h = self.norm2(x)
        x = x + self.mlp_down(F.gelu(self.mlp_up(h)))
        return x


class SimpleTransformer(nn.Module):
    """Simplified Transformer for baseline profiling."""

    def __init__(self, config: ModelConfig, num_layers: int = None):
        super().__init__()
        n = num_layers or config.num_layers
        self.embed = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.layers = nn.ModuleList([SimpleTransformerLayer(config) for _ in range(n)])
        self.norm = nn.LayerNorm(config.hidden_dim)
        self.head = nn.Linear(config.hidden_dim, config.vocab_size, bias=False)

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        x = self.embed(input_ids)
        for layer in self.layers:
            x = layer(x)
        x = self.norm(x)
        return self.head(x)

In [ ]:
# Profile the baseline: measure actual single-GPU memory and compute time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Use a smaller layer count for single-GPU profiling
profile_cfg = ModelConfig(num_layers=4, max_seq_len=2048, hidden_dim=1024, num_heads=8, head_dim=128, ffn_dim=2752)
model_profile = SimpleTransformer(profile_cfg, num_layers=4).to(device).bfloat16()

param_count = sum(p.numel() for p in model_profile.parameters())
print(f"Profile model: {param_count / 1e6:.1f}M parameters")

# Warm up
dummy = torch.randint(0, profile_cfg.vocab_size, (1, 512), device=device)
_ = model_profile(dummy)

# Measure forward + backward time and memory
torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

batch = torch.randint(0, profile_cfg.vocab_size, (2, 1024), device=device)
start = time.time()
logits = model_profile(batch)
loss = F.cross_entropy(logits.view(-1, profile_cfg.vocab_size), batch.view(-1))
loss.backward()
torch.cuda.synchronize()
elapsed = time.time() - start

peak_mem = torch.cuda.max_memory_allocated() / 1e9
print(f"Forward + backward time: {elapsed * 1000:.1f} ms")
print(f"Peak GPU memory: {peak_mem:.2f} GB")
print(f"Loss: {loss.item():.4f}")

# Estimate MFU
flops_per_token = 6 * param_count  # 6 = 2 (fwd) * 3 (fwd+bwd)
total_tokens = 2 * 1024  # batch_size * seq_len
total_flops = flops_per_token * total_tokens
peak_flops = hw_cfg.peak_tflops * 1e12  # Convert TFLOPS to FLOPS
mfu = total_flops / (elapsed * peak_flops)
print(f"Estimated MFU: {mfu * 100:.1f}%")

In [ ]:
# TODO: Estimate baseline FSDP throughput at the full scale (1,024 GPUs)

def estimate_fsdp_throughput(
    model: ModelConfig,
    hw: HardwareConfig,
    global_batch_size: int,
    seq_length: int,
    zero_stage: int = 3
) -> Dict[str, float]:
    """
    Estimate the throughput of pure FSDP (Data Parallelism only) at full scale.

    This function should compute:
    1. Compute time per step (based on FLOPs and peak GPU throughput)
    2. Communication time (AllGather + ReduceScatter for ZeRO-3)
    3. Total step time and tokens per second
    4. MFU

    Args:
        model: Model configuration
        hw: Hardware configuration
        global_batch_size: Total batch size across all GPUs
        seq_length: Sequence length
        zero_stage: ZeRO stage (0-3)

    Returns:
        Dictionary with:
        - 'compute_time_s': Time spent on computation
        - 'comm_time_s': Time spent on communication
        - 'step_time_s': Total step time
        - 'tokens_per_sec': Training throughput
        - 'mfu': Model FLOPs Utilization

    Hints:
        1. Compute FLOPs per step: 6 * active_params * global_batch_size * seq_length
        2. Compute time = FLOPs / (num_gpus * peak_tflops * 1e12)
           (This gives the ideal compute time; actual MFU will be lower)
        3. For ZeRO-3 communication: 2 AllGather + 2 ReduceScatter per step
           Each moves total_params * 2 bytes (BF16) through the network.
           The effective bandwidth for AllReduce across N GPUs is:
           bw_effective = 2 * (N-1)/N * per_link_bw * num_links_per_node / 2
           (The factor of 2*(N-1)/N accounts for the ring AllReduce algorithm)
        4. MFU = useful_compute_time / (step_time * num_gpus * peak_tflops * 1e12)
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement FSDP throughput estimation")


# Test at full scale
fsdp_perf = estimate_fsdp_throughput(model_cfg, hw_cfg, global_batch_size=256, seq_length=4096)
print("=== Baseline FSDP Performance (Full Scale) ===")
for key, val in fsdp_perf.items():
    if 'time' in key:
        print(f"  {key}: {val * 1000:.1f} ms")
    elif key == 'mfu':
        print(f"  {key}: {val * 100:.1f}%")
    else:
        print(f"  {key}: {val:,.0f}")

**Thought Questions:**
1. Why does FSDP's communication overhead become a bottleneck at 1,024 GPUs? (Hint: think about the ratio of compute to communication as the number of GPUs grows.)
2. What happens to MFU if we double the batch size? Why?
3. Why can FSDP not reduce activation memory, even though it shards weights and optimizer states?

---

## 3.4 Model Design: Configuring 5D Parallelism

### 3.4.1 Tensor Parallelism

Tensor Parallelism splits individual weight matrices across GPUs within a node. We implement the two fundamental building blocks: column-parallel and row-parallel linear layers.

In [ ]:
# TODO: Implement Column-Parallel Linear Layer

class ColumnParallelLinear(nn.Module):
    """
    Linear layer with weight matrix split column-wise across TP ranks.

    Given Y = XW, we split W = [W_0 | W_1 | ... | W_{tp-1}] along columns.
    Each TP rank i computes Y_i = X @ W_i (a partial output).
    The full output is obtained by concatenating: Y = [Y_0 | Y_1 | ... | Y_{tp-1}].

    For the Megatron-LM MLP, this is used for the FIRST linear layer (gate/up projection).
    The key insight: we do NOT need to concatenate explicitly if the next operation
    (GeLU) is element-wise — each rank can apply GeLU to its local output independently.

    Args:
        in_features: Input dimension (not split)
        out_features: Output dimension (split across TP ranks)
        tp_rank: This rank's position in the TP group (0 to tp_world_size-1)
        tp_world_size: Number of TP ranks
        bias: Whether to include a bias term

    Hints:
        1. The local output dimension is out_features // tp_world_size
        2. Initialize weight of shape (in_features, local_out_features)
        3. In forward(), simply compute x @ weight (+ bias if applicable)
        4. The output has shape (batch, seq, local_out_features)
    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        tp_rank: int = 0,
        tp_world_size: int = 1,
        bias: bool = False,
    ):
        super().__init__()
        self.tp_rank = tp_rank
        self.tp_world_size = tp_world_size
        # YOUR CODE HERE: Define the weight parameter with the correct shape
        # Remember: out_features is split across TP ranks
        raise NotImplementedError("Define weight parameter")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass: x @ weight.

        Args:
            x: Input tensor of shape (batch, seq, in_features)

        Returns:
            Output tensor of shape (batch, seq, local_out_features)
        """
        # YOUR CODE HERE
        raise NotImplementedError("Implement forward pass")

In [ ]:
# TODO: Implement Row-Parallel Linear Layer

class RowParallelLinear(nn.Module):
    """
    Linear layer with weight matrix split row-wise across TP ranks.

    Given Y = XW, we split W along rows: W = [W_0; W_1; ...; W_{tp-1}].
    The input X must also be split: X = [X_0 | X_1 | ... | X_{tp-1}].
    Each TP rank i computes Y_i = X_i @ W_i (a partial sum).
    The full output is: Y = sum(Y_0, Y_1, ..., Y_{tp-1}) via AllReduce.

    For the Megatron-LM MLP, this is used for the SECOND linear layer (down projection).
    The input to this layer is already split across TP ranks (output of ColumnParallel + GeLU),
    so no explicit input splitting is needed.

    Args:
        in_features: Input dimension (split across TP ranks)
        out_features: Output dimension (not split)
        tp_rank: This rank's position in the TP group
        tp_world_size: Number of TP ranks
        bias: Whether to include a bias term

    Hints:
        1. The local input dimension is in_features // tp_world_size
        2. Weight shape is (local_in_features, out_features)
        3. In forward(), compute x @ weight — this gives a partial result
        4. In a real distributed setup, you would AllReduce the partial results.
           For this single-GPU implementation, just return the partial result.
           We will simulate the AllReduce in the combined MLP module.
    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        tp_rank: int = 0,
        tp_world_size: int = 1,
        bias: bool = False,
    ):
        super().__init__()
        self.tp_rank = tp_rank
        self.tp_world_size = tp_world_size
        # YOUR CODE HERE: Define the weight parameter with the correct shape
        raise NotImplementedError("Define weight parameter")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass: x @ weight (partial result, needs AllReduce for full output).

        Args:
            x: Input tensor of shape (batch, seq, local_in_features)

        Returns:
            Partial output tensor of shape (batch, seq, out_features)
        """
        # YOUR CODE HERE
        raise NotImplementedError("Implement forward pass")

In [ ]:
# TODO: Implement the Megatron-LM Tensor-Parallel MLP

class TensorParallelMLP(nn.Module):
    """
    Megatron-LM style MLP with Tensor Parallelism.

    Architecture:
        Input -> ColumnParallelLinear (gate_proj) -> GeLU -> RowParallelLinear (down_proj) -> Output
                                                                    ^
                                                              AllReduce here

    The beauty of this design:
    - ColumnParallel splits the output, so each rank has a different slice
    - GeLU is element-wise, so it can be applied locally (NO communication)
    - RowParallel takes the split input and produces partial sums
    - Only ONE AllReduce is needed (after RowParallel), not two

    This is the key insight from the Megatron-LM paper (Shoeybi et al., 2019).

    Args:
        hidden_dim: Model hidden dimension
        ffn_dim: MLP intermediate dimension
        tp_rank: TP rank
        tp_world_size: TP world size

    Hints:
        1. gate_proj: ColumnParallelLinear(hidden_dim, ffn_dim, ...)
        2. down_proj: RowParallelLinear(ffn_dim, hidden_dim, ...)
        3. Forward: x -> gate_proj -> gelu -> down_proj -> (allreduce) -> output
        4. Simulate AllReduce by summing partial results across TP ranks
    """

    def __init__(self, hidden_dim: int, ffn_dim: int, tp_rank: int = 0, tp_world_size: int = 1):
        super().__init__()
        # YOUR CODE HERE: Initialize gate_proj and down_proj
        raise NotImplementedError("Initialize MLP layers")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass with Megatron-style TP.

        Args:
            x: Input of shape (batch, seq, hidden_dim)

        Returns:
            Output of shape (batch, seq, hidden_dim)
        """
        # YOUR CODE HERE
        raise NotImplementedError("Implement Megatron MLP forward")

In [ ]:
# Verification: Test Tensor Parallel MLP
# Create TP=2 MLPs and verify their outputs match a single-GPU MLP

torch.manual_seed(42)
H, FFN = 256, 512
x_test = torch.randn(2, 16, H, device=device, dtype=torch.bfloat16)

# Single-GPU reference
mlp_full = nn.Sequential(
    nn.Linear(H, FFN, bias=False),
    nn.GELU(),
    nn.Linear(FFN, H, bias=False)
).to(device).bfloat16()

y_ref = mlp_full(x_test)

# TP=2 version (simulated on single GPU)
mlp_tp0 = TensorParallelMLP(H, FFN, tp_rank=0, tp_world_size=2).to(device).bfloat16()
mlp_tp1 = TensorParallelMLP(H, FFN, tp_rank=1, tp_world_size=2).to(device).bfloat16()

# Copy weights from reference to TP shards
with torch.no_grad():
    # Column split for gate_proj
    half_ffn = FFN // 2
    mlp_tp0.gate_proj.weight.copy_(mlp_full[0].weight[:half_ffn, :].t())  # Adjust based on your impl
    mlp_tp1.gate_proj.weight.copy_(mlp_full[0].weight[half_ffn:, :].t())
    # Row split for down_proj
    mlp_tp0.down_proj.weight.copy_(mlp_full[2].weight[:, :half_ffn].t())
    mlp_tp1.down_proj.weight.copy_(mlp_full[2].weight[:, half_ffn:].t())

# Compute TP outputs and simulate AllReduce (sum)
y_tp0 = mlp_tp0(x_test)
y_tp1 = mlp_tp1(x_test)
y_tp = y_tp0 + y_tp1  # Simulated AllReduce

print(f"Reference output shape: {y_ref.shape}")
print(f"TP output shape: {y_tp.shape}")
print(f"Max difference: {(y_ref - y_tp).abs().max().item():.6f}")
print(f"Match (tolerance 1e-2): {torch.allclose(y_ref, y_tp, atol=1e-2)}")

**Thought Questions:**
1. Why must Tensor Parallelism stay within a single node (NVLink boundary)? What would happen to throughput if TP spanned across nodes using InfiniBand?
2. The Megatron design achieves ONE AllReduce per MLP block. Could you design a scheme with ZERO AllReduces? Why or why not?
3. How does Tensor Parallelism affect the memory consumed by activations? (Hint: consider what each GPU stores after the column-parallel split.)

### 3.4.2 Pipeline Parallelism

Pipeline Parallelism splits the model's layers across GPUs in sequence, like an assembly line. The key challenge is minimizing the "pipeline bubble" — the fraction of time GPUs sit idle.

In [ ]:
# TODO: Implement a Pipeline Stage

class PipelineStage:
    """
    Represents one stage of the pipeline, holding a subset of the model's layers.

    In production Pipeline Parallelism (e.g., Megatron-LM), each stage:
    1. Receives activations from the previous stage (or input embeddings for stage 0)
    2. Runs forward through its local layers
    3. Sends activations to the next stage (or computes loss for the last stage)

    For the backward pass, gradients flow in reverse.

    Args:
        layers: List of nn.Module layers assigned to this stage
        stage_id: This stage's index (0 to num_stages-1)
        num_stages: Total number of pipeline stages

    Hints:
        1. Store the layers as an nn.ModuleList
        2. forward() runs input through each layer sequentially
        3. Track whether this is the first stage (receives input tokens)
           or last stage (computes loss)
    """

    def __init__(self, layers: List[nn.Module], stage_id: int, num_stages: int):
        # YOUR CODE HERE
        raise NotImplementedError("Initialize pipeline stage")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Run forward pass through this stage's layers."""
        # YOUR CODE HERE
        raise NotImplementedError("Implement stage forward")

    def backward(self, grad: torch.Tensor):
        """Run backward pass (gradient flows in reverse through layers)."""
        # YOUR CODE HERE
        raise NotImplementedError("Implement stage backward")

In [ ]:
# TODO: Implement the 1F1B Pipeline Schedule

def one_f_one_b_schedule(
    num_stages: int,
    num_microbatches: int
) -> List[List[Tuple[str, int]]]:
    """
    Generate the 1F1B (one-forward-one-backward) pipeline schedule.

    The schedule tells each stage WHEN to run a forward (F) or backward (B)
    pass for each micro-batch. The 1F1B schedule interleaves F and B to
    reduce peak memory (activations are released earlier).

    The schedule has three phases:
    1. Warmup: Each stage runs forward passes to fill the pipeline
    2. Steady state: Alternating 1 forward + 1 backward per step
    3. Cooldown: Remaining backward passes drain the pipeline

    Args:
        num_stages: Number of pipeline stages (P)
        num_microbatches: Number of micro-batches (M)

    Returns:
        Schedule as a list of lists. schedule[stage_id] is a list of
        (action, microbatch_id) tuples, where action is 'F' or 'B'.

    Example for P=4, M=8:
        Stage 0: [F0, F1, F2, F3, B0, F4, B1, F5, B2, F6, B3, F7, B4, B5, B6, B7]
        Stage 1: [__, F0, F1, F2, F3, B0, F4, B1, F5, B2, F6, B3, F7, B4, B5, B6]
        ...

    Hints:
        1. Warmup phase for stage s: run (num_stages - 1 - s) forward passes
        2. Steady state: alternate F and B for the remaining micro-batches
        3. Cooldown: run remaining backward passes
        4. Count the idle slots ('__') — this is the bubble
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement 1F1B schedule")


# Compute the bubble fraction
def bubble_fraction(num_stages: int, num_microbatches: int) -> float:
    """
    Compute the pipeline bubble fraction.

    Formula: (P - 1) / (P - 1 + M)

    Args:
        num_stages: P
        num_microbatches: M

    Returns:
        Bubble fraction (0 to 1)
    """
    return (num_stages - 1) / (num_stages - 1 + num_microbatches)

In [ ]:
# Verification: Visualize the pipeline schedule and bubble

def visualize_schedule(schedule: List[List[Tuple[str, int]]], num_stages: int):
    """Print a text-based visualization of the pipeline schedule."""
    if schedule is None:
        print("Schedule not implemented yet")
        return

    max_steps = max(len(stage_schedule) for stage_schedule in schedule)
    print(f"\n{'Stage':<8}", end='')
    for t in range(max_steps):
        print(f"{'t=' + str(t):<6}", end='')
    print()
    print('-' * (8 + max_steps * 6))

    for s in range(num_stages):
        print(f"GPU {s:<4}", end='')
        for t in range(max_steps):
            if t < len(schedule[s]):
                action, mb = schedule[s][t]
                print(f"{action}{mb:<5}", end='')
            else:
                print(f"{'--':<6}", end='')
        print()

P, M = 4, 8
schedule = one_f_one_b_schedule(P, M)
visualize_schedule(schedule, P)
print(f"\nBubble fraction: {bubble_fraction(P, M) * 100:.1f}%")

# Plot bubble fraction vs number of micro-batches
microbatches = list(range(1, 33))
fig, ax = plt.subplots(figsize=(8, 4))
for p in [2, 4, 8, 16]:
    bubbles = [bubble_fraction(p, m) * 100 for m in microbatches]
    ax.plot(microbatches, bubbles, label=f'P={p} stages')
ax.set_xlabel('Number of micro-batches (M)')
ax.set_ylabel('Bubble fraction (%)')
ax.set_title('Pipeline Bubble vs Micro-batches')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.show()

**Thought Questions:**
1. The bubble fraction formula is $(P-1)/(P-1+M)$. Derive this from first principles by counting the idle slots in the pipeline.
2. Why does 1F1B use less peak memory than GPipe (all-forward-then-all-backward)?
3. What is the tradeoff between increasing the number of micro-batches $M$ and the effective batch size? (Hint: $M$ micro-batches means each has $B_{\text{global}} / (M \times N_{\text{DP}})$ sequences.)

### 3.4.3 Sequence Parallelism (Context Parallelism)

For long sequences (128K+ tokens), the attention score matrix becomes too large for a single GPU. Ring Attention distributes the attention computation across GPUs arranged in a ring.

In [ ]:
# TODO: Implement simplified Ring Attention

class RingAttention(nn.Module):
    """
    Simplified Ring Attention for Context Parallelism.

    The idea: split Q, K, V along the sequence dimension across GPUs in a ring.
    Each GPU holds a local chunk of Q and iterates through all K, V chunks by
    passing them around the ring.

    For this single-GPU simulation, we split the sequence into chunks and
    simulate the ring communication by iterating through the chunks.

    Steps for each GPU (rank r, ring of size R):
    1. Start with local Q_r, K_r, V_r (each has seq_len/R tokens)
    2. For step s = 0, 1, ..., R-1:
       a. Compute attention: A_s = softmax(Q_r @ K_{(r+s)%R}^T / sqrt(d)) @ V_{(r+s)%R}
       b. Accumulate the output using the log-sum-exp trick for numerical stability
       c. Pass K, V to the next GPU in the ring (receive from previous)
    3. Final output O_r is the correctly weighted combination of all partial attentions

    Args:
        hidden_dim: Model hidden dimension
        num_heads: Number of attention heads
        head_dim: Dimension per head
        cp_world_size: Context Parallelism (ring) size

    Hints:
        1. Split Q, K, V into cp_world_size chunks along the sequence dimension
        2. For each ring step, compute attention between local Q and current K, V
        3. Use the online softmax trick: track running max and running sum of exp
           to numerically combine attention outputs from different K, V chunks
        4. The final output for each Q chunk should be identical to full attention
    """

    def __init__(self, hidden_dim: int, num_heads: int, head_dim: int, cp_world_size: int = 1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.cp_world_size = cp_world_size
        self.scale = head_dim ** -0.5

    def forward(
        self,
        q: torch.Tensor,
        k: torch.Tensor,
        v: torch.Tensor,
    ) -> torch.Tensor:
        """
        Compute Ring Attention.

        Args:
            q: Query tensor of shape (batch, num_heads, seq_len, head_dim)
            k: Key tensor of shape (batch, num_heads, seq_len, head_dim)
            v: Value tensor of shape (batch, num_heads, seq_len, head_dim)

        Returns:
            Output tensor of shape (batch, num_heads, seq_len, head_dim)
        """
        B, H, S, D = q.shape
        R = self.cp_world_size

        if R == 1:
            # No Context Parallelism — standard attention
            return F.scaled_dot_product_attention(q, k, v)

        chunk_size = S // R
        # Split Q into chunks (one per ring rank)
        q_chunks = q.split(chunk_size, dim=2)  # R chunks of (B, H, S/R, D)
        k_chunks = k.split(chunk_size, dim=2)
        v_chunks = v.split(chunk_size, dim=2)

        outputs = []

        # Simulate each ring rank
        for rank in range(R):
            # YOUR CODE HERE:
            # For this rank's Q chunk, compute attention against ALL K, V chunks
            # by simulating the ring rotation.
            #
            # Use the online softmax (log-sum-exp) trick:
            #   - Track running_max (max of attention logits seen so far)
            #   - Track running_sum (sum of exp(logits - running_max))
            #   - Track running_output (weighted sum of V)
            #   - For each new K, V chunk:
            #       1. Compute logits = Q_local @ K_chunk^T * scale
            #       2. new_max = max(running_max, logits.max)
            #       3. Rescale running terms: multiply by exp(running_max - new_max)
            #       4. Add new contribution: exp(logits - new_max) @ V_chunk
            #       5. Update running_max, running_sum, running_output
            #   - Final output = running_output / running_sum
            raise NotImplementedError("Implement ring attention for one rank")

        # Concatenate outputs from all ranks
        output = torch.cat(outputs, dim=2)  # (B, H, S, D)
        return output

In [ ]:
# Verification: Ring Attention should produce identical results to standard attention

torch.manual_seed(42)
B, H, S, D = 1, 4, 256, 64
q = torch.randn(B, H, S, D, device=device, dtype=torch.bfloat16)
k = torch.randn(B, H, S, D, device=device, dtype=torch.bfloat16)
v = torch.randn(B, H, S, D, device=device, dtype=torch.bfloat16)

# Reference: standard attention
ref_out = F.scaled_dot_product_attention(q, k, v)

# Ring Attention with CP=4
ring_attn = RingAttention(H * D, H, D, cp_world_size=4).to(device)
ring_out = ring_attn(q, k, v)

print(f"Reference output shape: {ref_out.shape}")
print(f"Ring Attention output shape: {ring_out.shape}")
print(f"Max difference: {(ref_out - ring_out).abs().max().item():.6f}")
print(f"Match (tolerance 1e-2): {torch.allclose(ref_out, ring_out, atol=1e-2)}")

# Memory comparison
full_attn_mem = B * H * S * S * 2 / 1e6  # Full attention score matrix in BF16
ring_attn_mem = B * H * (S // 4) * (S // 4) * 2 / 1e6  # One chunk at a time
print(f"\nFull attention memory: {full_attn_mem:.1f} MB")
print(f"Ring attention memory (per step): {ring_attn_mem:.1f} MB")
print(f"Memory reduction: {full_attn_mem / ring_attn_mem:.1f}x")

**Thought Questions:**
1. Ring Attention trades memory for communication. How many bytes of K, V data does each GPU send per ring step? For 128K tokens with 8192 hidden dim, what is the total communication volume?
2. Why is the online softmax trick necessary? What would happen if you naively averaged the attention outputs from each ring step?
3. In practice, the ring communication (sending K, V) can be overlapped with the attention computation. Draw a timeline showing how this overlap works for 4 ring steps.

### 3.4.4 Expert Parallelism

Expert Parallelism distributes MoE experts across GPUs. Each GPU hosts a subset of experts and processes only the tokens routed to its experts.

In [ ]:
# TODO: Implement Expert-Parallel MoE Layer

class ExpertParallelMoE(nn.Module):
    """
    Mixture-of-Experts layer with Expert Parallelism.

    Components:
    1. Router: A linear layer that produces gate logits for each token
    2. Top-K selection: Select the top-K experts for each token
    3. All-to-All dispatch: Send tokens to the GPU hosting their assigned expert
    4. Expert computation: Each GPU runs its local experts on received tokens
    5. All-to-All collect: Send results back to the original GPU
    6. Weighted combination: Combine expert outputs using gate weights

    For this single-GPU simulation, we implement all experts locally but simulate
    the All-to-All communication cost and load balancing.

    Args:
        hidden_dim: Model hidden dimension
        expert_ffn_dim: Per-expert MLP intermediate dimension
        num_experts: Total number of experts
        top_k: Number of experts activated per token
        ep_world_size: Expert Parallelism degree (experts per GPU = num_experts / ep_world_size)

    Hints:
        1. Router: nn.Linear(hidden_dim, num_experts)
        2. Top-K: Use torch.topk on gate logits, then softmax to normalize
        3. Expert MLP: nn.Linear(hidden_dim, expert_ffn_dim) + GeLU + nn.Linear(expert_ffn_dim, hidden_dim)
        4. Load balancing loss: alpha * N_experts * sum(f_i * p_i)
           where f_i = fraction of tokens routed to expert i
           and p_i = mean gate probability for expert i
    """

    def __init__(
        self,
        hidden_dim: int,
        expert_ffn_dim: int,
        num_experts: int = 8,
        top_k: int = 2,
        ep_world_size: int = 1,
        balance_loss_weight: float = 0.01,
    ):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.ep_world_size = ep_world_size
        self.experts_per_gpu = num_experts // ep_world_size
        self.balance_loss_weight = balance_loss_weight

        # YOUR CODE HERE:
        # 1. Define the router (linear layer: hidden_dim -> num_experts)
        # 2. Define expert MLPs (one for each expert)
        #    Each expert: Linear(hidden_dim, expert_ffn_dim) -> GeLU -> Linear(expert_ffn_dim, hidden_dim)
        #    Use nn.ModuleList to hold all experts
        raise NotImplementedError("Initialize MoE components")

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass through the MoE layer.

        Args:
            x: Input tensor of shape (batch, seq, hidden_dim)

        Returns:
            Tuple of:
            - Output tensor of shape (batch, seq, hidden_dim)
            - Auxiliary load-balancing loss (scalar)

        Steps:
            1. Compute gate logits: router(x) -> (batch, seq, num_experts)
            2. Apply softmax to get gate probabilities
            3. Select top-K experts per token
            4. Renormalize top-K gate weights to sum to 1
            5. For each token, compute weighted sum of selected expert outputs
            6. Compute load-balancing loss

        Hints for load-balancing loss:
            - f_i = (number of tokens where expert i is in top-K) / total_tokens
            - p_i = mean of gate_probs[:, :, i] across all tokens
            - loss = balance_loss_weight * num_experts * sum(f_i * p_i)
        """
        # YOUR CODE HERE
        raise NotImplementedError("Implement MoE forward pass")

In [ ]:
# Verification: Test MoE layer and check load balancing

torch.manual_seed(42)
moe = ExpertParallelMoE(
    hidden_dim=256,
    expert_ffn_dim=512,
    num_experts=8,
    top_k=2,
    ep_world_size=1,
    balance_loss_weight=0.01
).to(device).bfloat16()

x_test = torch.randn(2, 64, 256, device=device, dtype=torch.bfloat16)
output, aux_loss = moe(x_test)

print(f"Input shape: {x_test.shape}")
print(f"Output shape: {output.shape}")
print(f"Auxiliary loss: {aux_loss.item():.6f}")

# Check expert utilization
with torch.no_grad():
    gate_logits = moe.router(x_test.float())
    gate_probs = F.softmax(gate_logits, dim=-1)
    top_k_indices = torch.topk(gate_probs, k=2, dim=-1).indices  # (B, S, K)

    expert_counts = torch.zeros(8)
    for i in range(8):
        expert_counts[i] = (top_k_indices == i).sum().item()

    print(f"\nExpert utilization (token counts):")
    for i in range(8):
        bar = '#' * int(expert_counts[i] / 5)
        print(f"  Expert {i}: {expert_counts[i]:5.0f} tokens  {bar}")

    max_load = expert_counts.max().item()
    mean_load = expert_counts.mean().item()
    imbalance = max_load / mean_load if mean_load > 0 else float('inf')
    print(f"\nLoad imbalance (max/mean): {imbalance:.2f}")
    print(f"Target: <= 1.10")

**Thought Questions:**
1. Why does Expert Parallelism require All-to-All communication (not AllReduce)? How does the communication pattern differ?
2. If you disable the load-balancing loss, what happens to expert utilization over many training steps? Why?
3. DeepSeek-V3 uses 256 experts with top-8 routing. How many GPUs would you need for Expert Parallelism if each GPU hosts 8 experts? What is the All-to-All communication cost?

### 3.4.5 Combining All Five Dimensions

Now we bring everything together: find the optimal 5D parallelism configuration that maximizes throughput while fitting within the memory budget.

In [ ]:
# TODO: Implement the parallelism configuration optimizer

def estimate_throughput(
    model: ModelConfig,
    hw: HardwareConfig,
    par: ParallelismConfig,
    global_batch_size: int,
    seq_length: int,
) -> Dict[str, float]:
    """
    Estimate training throughput for a given 5D parallelism configuration.

    This combines the memory model and throughput model from Section 2.

    Args:
        model: Model configuration
        hw: Hardware configuration
        par: Parallelism configuration
        global_batch_size: Total batch size
        seq_length: Sequence length

    Returns:
        Dictionary with:
        - 'memory_gb': Peak per-GPU memory (GB)
        - 'fits_in_memory': Whether config fits in GPU memory
        - 'compute_time_s': Compute time per step
        - 'tp_comm_time_s': Tensor Parallel communication time
        - 'pp_comm_time_s': Pipeline Parallel communication time
        - 'dp_comm_time_s': Data Parallel communication time
        - 'ep_comm_time_s': Expert Parallel communication time
        - 'bubble_time_s': Pipeline bubble time
        - 'step_time_s': Total step time
        - 'tokens_per_sec': Throughput in tokens/second
        - 'mfu': Model FLOPs Utilization

    Hints:
        1. First check memory: call estimate_memory_bytes and verify it fits
        2. Compute FLOPs per step: 6 * active_params * global_batch_size * seq_length
        3. Compute time = FLOPs / (num_gpus * peak_tflops * 1e12)
        4. TP comm: 2 AllReduce per layer * layers_per_stage / NVLink bandwidth
           Message size per AllReduce: batch * seq * hidden * 2 bytes (BF16)
        5. PP comm: point-to-point activation transfer at stage boundaries
           Message size: micro_batch * seq * hidden * 2 bytes
           Over InfiniBand bandwidth
        6. DP comm: 1 AllReduce of gradients per step
           Message size: params_per_gpu * 2 bytes / dp group size (ring AllReduce)
           Over InfiniBand
        7. EP comm: All-to-All for each MoE layer
           Message size: batch * seq * hidden * 2 bytes * (ep - 1) / ep
        8. Bubble: bubble_fraction(pp, num_microbatches) * compute_time
        9. MFU = useful_flops / (step_time * num_gpus * peak_tflops * 1e12)
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement throughput estimation")


def find_optimal_config(
    model: ModelConfig,
    hw: HardwareConfig,
    seq_length: int,
    global_batch_size: int = 256,
) -> Tuple[ParallelismConfig, Dict[str, float]]:
    """
    Search for the optimal 5D parallelism configuration.

    Enumerate all valid configurations and return the one with highest throughput
    that fits within the GPU memory budget.

    Args:
        model: Model configuration
        hw: Hardware configuration
        seq_length: Target sequence length
        global_batch_size: Target global batch size

    Returns:
        Tuple of (best config, performance metrics)

    Hints:
        1. Generate all valid (DP, TP, PP, EP) tuples where:
           - DP * TP * PP * EP = num_gpus
           - TP in {1, 2, 4, 8} and TP <= gpus_per_node
           - PP divides num_layers evenly
           - EP divides num_experts evenly
        2. For each config, try different ZeRO stages (0, 1, 2, 3)
        3. For each config, try activation_checkpointing on/off
        4. Set num_microbatches = max(4, global_batch_size // (DP * micro_batch_size))
        5. Evaluate throughput and track the best
        6. Return the configuration with highest MFU among those that fit in memory
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement configuration search")

In [ ]:
# Run the optimizer
print("Searching for optimal parallelism configuration...")
print(f"Model: {model_cfg.total_params / 1e9:.2f}B params, {model_cfg.num_layers} layers, {model_cfg.num_experts} experts")
print(f"Hardware: {hw_cfg.num_gpus} GPUs, {hw_cfg.gpus_per_node} per node")
print(f"Target: seq_len={model_cfg.max_seq_len}, batch_size=256\n")

best_config, best_perf = find_optimal_config(model_cfg, hw_cfg, seq_length=model_cfg.max_seq_len)

print("=== Optimal Configuration ===")
print(f"  DP = {best_config.dp}")
print(f"  TP = {best_config.tp}")
print(f"  PP = {best_config.pp}")
print(f"  SP = {best_config.sp} (= TP)")
print(f"  EP = {best_config.ep}")
print(f"  ZeRO Stage = {best_config.zero_stage}")
print(f"  Activation Checkpointing = {best_config.activation_checkpointing}")
print(f"  Micro-batches = {best_config.num_microbatches}")
print(f"\n=== Performance Metrics ===")
for key, val in best_perf.items():
    if 'time' in key:
        print(f"  {key}: {val * 1000:.1f} ms")
    elif key == 'mfu':
        print(f"  {key}: {val * 100:.1f}%")
    elif key == 'memory_gb':
        print(f"  {key}: {val:.1f} GB / {hw_cfg.gpu_memory_gb:.0f} GB")
    elif key == 'fits_in_memory':
        print(f"  {key}: {val}")
    else:
        print(f"  {key}: {val:,.0f}")

In [ ]:
# Visualization: Compare top 5 configurations

def compare_top_configs(model, hw, seq_length, global_batch_size=256, top_n=5):
    """Find and visualize the top-N parallelism configurations."""
    configs = []
    tp_options = [t for t in [1, 2, 4, 8] if t <= hw.gpus_per_node]
    pp_options = [p for p in [1, 2, 4, 8] if model.num_layers % p == 0]
    ep_options = [e for e in [1, 2, 4, 8] if model.num_experts % e == 0]

    for tp in tp_options:
        for pp in pp_options:
            for ep in ep_options:
                dp = hw.num_gpus // (tp * pp * ep)
                if dp < 1 or dp * tp * pp * ep != hw.num_gpus:
                    continue
                for zero in [1, 3]:
                    for ac in [True, False]:
                        par = ParallelismConfig(
                            dp=dp, tp=tp, pp=pp, sp=tp, ep=ep,
                            zero_stage=zero,
                            activation_checkpointing=ac,
                            num_microbatches=max(4, global_batch_size // dp)
                        )
                        errors = par.validate(hw, model)
                        if errors:
                            continue
                        perf = estimate_throughput(model, hw, par, global_batch_size, seq_length)
                        if perf['fits_in_memory']:
                            configs.append((par, perf))

    # Sort by MFU
    configs.sort(key=lambda x: x[1]['mfu'], reverse=True)

    print(f"\nTop {top_n} Configurations:\n")
    print(f"{'Rank':<6}{'DP':<5}{'TP':<5}{'PP':<5}{'EP':<5}{'ZeRO':<6}{'AC':<5}{'Memory (GB)':<13}{'MFU (%)':<10}{'Tok/s':<12}")
    print('-' * 76)
    for i, (par, perf) in enumerate(configs[:top_n]):
        print(f"{i+1:<6}{par.dp:<5}{par.tp:<5}{par.pp:<5}{par.ep:<5}{par.zero_stage:<6}{'Y' if par.activation_checkpointing else 'N':<5}{perf['memory_gb']:<13.1f}{perf['mfu']*100:<10.1f}{perf['tokens_per_sec']:<12,.0f}")

    return configs[:top_n]

top_configs = compare_top_configs(model_cfg, hw_cfg, model_cfg.max_seq_len)

**Thought Questions:**
1. Why does the optimal configuration typically have TP=8 (full intra-node)? Under what circumstances might TP=4 or TP=2 be better?
2. How does the Expert Parallelism degree interact with Pipeline Parallelism? Can you have both EP=8 and PP=8 on a 1024-GPU cluster? Why or why not?
3. For the Meridian production model (72B params, 128K seq length, 1024 GPUs), estimate the optimal configuration by scaling up from your small-model results. What changes when you scale by 50x in parameters?

---

## 3.5 Training Strategy

In [ ]:
# Training loop integrating all 5 parallelism dimensions (single-GPU simulation)

def distributed_training_step(
    model: nn.Module,
    batch: torch.Tensor,
    optimizer: torch.optim.Optimizer,
    config: ParallelismConfig,
    model_cfg: ModelConfig,
    grad_clip_norm: float = 1.0,
) -> Dict[str, float]:
    """
    Simulate one distributed training step with 5D parallelism.

    On a single GPU, we simulate the effects of parallelism:
    - TP: Compute with reduced hidden dimension (simulated split)
    - PP: Process only a subset of layers (num_layers // pp)
    - DP: Process 1/dp of the batch
    - EP: Process tokens only through top-K of local experts
    - SP: Process 1/sp of the sequence

    In production, this would use torch.distributed with NCCL collectives.

    Args:
        model: The model (full or subset depending on simulation)
        batch: Input token IDs
        optimizer: Optimizer (AdamW)
        config: Parallelism configuration
        model_cfg: Model configuration
        grad_clip_norm: Max gradient norm for clipping

    Returns:
        Dictionary with loss, gradient norm, step time, and simulated MFU
    """
    model.train()
    optimizer.zero_grad()

    start_time = time.time()

    # Forward pass
    logits = model(batch)
    targets = batch[:, 1:].contiguous()
    logits = logits[:, :-1].contiguous()
    loss = F.cross_entropy(logits.view(-1, model_cfg.vocab_size), targets.view(-1))

    # Backward pass
    loss.backward()

    # Gradient clipping
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)

    # Optimizer step
    optimizer.step()

    step_time = time.time() - start_time

    return {
        'loss': loss.item(),
        'grad_norm': grad_norm.item() if isinstance(grad_norm, torch.Tensor) else grad_norm,
        'step_time': step_time,
    }

In [ ]:
# TODO: Implement a learning rate schedule with warmup and cosine decay

def get_cosine_schedule_with_warmup(
    optimizer: torch.optim.Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    min_lr_ratio: float = 0.1,
) -> torch.optim.lr_scheduler.LambdaLR:
    """
    Create a cosine learning rate schedule with linear warmup.

    The schedule:
    - Steps 0 to num_warmup_steps: Linear increase from 0 to base_lr
    - Steps num_warmup_steps to num_training_steps: Cosine decay from base_lr to base_lr * min_lr_ratio

    Args:
        optimizer: The optimizer
        num_warmup_steps: Number of warmup steps
        num_training_steps: Total training steps
        min_lr_ratio: Ratio of minimum LR to base LR (default 0.1)

    Returns:
        LambdaLR scheduler

    Hints:
        1. During warmup: lr_scale = step / num_warmup_steps
        2. After warmup: progress = (step - warmup) / (total - warmup)
           lr_scale = min_lr_ratio + 0.5 * (1 - min_lr_ratio) * (1 + cos(pi * progress))
        3. Create a lambda function and pass to LambdaLR
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement cosine schedule with warmup")

In [ ]:
# Run a short training loop

# Use a small model for actual training on Colab
train_cfg = ModelConfig(
    vocab_size=32000, hidden_dim=512, num_layers=6, num_heads=8,
    head_dim=64, ffn_dim=1376, num_experts=4, top_k=2,
    expert_ffn_dim=512, max_seq_len=512
)
train_model = SimpleTransformer(train_cfg, num_layers=6).to(device).bfloat16()
train_optimizer = torch.optim.AdamW(train_model.parameters(), lr=3e-4, weight_decay=0.1, betas=(0.9, 0.95))

num_steps = 100
scheduler = get_cosine_schedule_with_warmup(train_optimizer, num_warmup_steps=10, num_training_steps=num_steps)

# Generate synthetic training data
train_data = create_synthetic_dataset(512, train_cfg.max_seq_len, train_cfg.vocab_size)
train_batches = [train_data[i:i+4] for i in range(0, len(train_data), 4)]

losses = []
lrs = []
grad_norms = []
step_times = []

print("Training...")
par_cfg = ParallelismConfig(dp=1, tp=1, pp=1, sp=1, ep=1)

for step in range(min(num_steps, len(train_batches))):
    batch = train_batches[step].to(device)
    metrics = distributed_training_step(
        train_model, batch, train_optimizer, par_cfg, train_cfg
    )
    scheduler.step()

    losses.append(metrics['loss'])
    lrs.append(scheduler.get_last_lr()[0])
    grad_norms.append(metrics['grad_norm'])
    step_times.append(metrics['step_time'])

    if (step + 1) % 20 == 0:
        print(f"  Step {step+1}/{num_steps}: loss={metrics['loss']:.4f}, "
              f"grad_norm={metrics['grad_norm']:.4f}, "
              f"time={metrics['step_time']*1000:.1f}ms, "
              f"lr={lrs[-1]:.2e}")

# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(losses)
axes[0, 0].set_title('Training Loss')
axes[0, 0].set_xlabel('Step')
axes[0, 1].plot(lrs)
axes[0, 1].set_title('Learning Rate')
axes[0, 1].set_xlabel('Step')
axes[1, 0].plot(grad_norms)
axes[1, 0].set_title('Gradient Norm')
axes[1, 0].set_xlabel('Step')
axes[1, 1].plot([t * 1000 for t in step_times])
axes[1, 1].set_title('Step Time (ms)')
axes[1, 1].set_xlabel('Step')
plt.tight_layout()
plt.show()

**Thought Questions:**
1. Why do we use AdamW instead of Adam? What is the difference in how weight decay is applied, and why does it matter for large Transformers?
2. The cosine schedule decays to 10% of the peak LR. What would happen if it decayed to 0? What about 50%?
3. Why is gradient clipping especially important in pipeline-parallel training? (Hint: think about gradient magnitudes across different pipeline stages.)

---

## 3.6 Evaluation

In [ ]:
# TODO: Generate evaluation comparing baseline vs optimized configuration

def evaluate_configurations(
    model: ModelConfig,
    hw: HardwareConfig,
    baseline: ParallelismConfig,
    optimized: ParallelismConfig,
    seq_length: int,
    global_batch_size: int = 256,
) -> Dict[str, Dict[str, float]]:
    """
    Compare baseline (FSDP) and optimized 5D parallelism configurations.

    Compute all metrics for both configurations and return a structured comparison.

    Args:
        model: Model configuration
        hw: Hardware configuration
        baseline: Baseline parallelism config (pure FSDP)
        optimized: Optimized 5D parallelism config
        seq_length: Sequence length
        global_batch_size: Global batch size

    Returns:
        Dictionary with 'baseline' and 'optimized' sub-dictionaries,
        each containing all performance metrics.

    Hints:
        1. Call estimate_throughput for both configurations
        2. Also compute memory breakdown for both
        3. Compute pipeline bubble fraction for the optimized config
        4. Compute estimated training days: total_tokens / (tokens_per_sec * 86400)
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement configuration comparison")

In [ ]:
# Visualize the comparison

baseline_cfg = ParallelismConfig(dp=8, tp=1, pp=1, sp=1, ep=1, zero_stage=3)
# Use the optimal config found earlier (or a manually specified good config)
opt_cfg = best_config  # From the optimizer above

comparison = evaluate_configurations(
    model_cfg, hw_cfg, baseline_cfg, opt_cfg, seq_length=model_cfg.max_seq_len
)

# Print comparison table
print("=" * 70)
print(f"{'Metric':<30}{'Baseline (FSDP)':<20}{'Optimized 5D':<20}")
print("=" * 70)
for key in comparison['baseline']:
    bval = comparison['baseline'][key]
    oval = comparison['optimized'][key]
    if isinstance(bval, bool):
        print(f"{key:<30}{str(bval):<20}{str(oval):<20}")
    elif 'time' in key:
        print(f"{key:<30}{bval*1000:>15.1f} ms  {oval*1000:>15.1f} ms")
    elif key == 'mfu':
        print(f"{key:<30}{bval*100:>15.1f}%    {oval*100:>15.1f}%")
    elif key == 'memory_gb':
        print(f"{key:<30}{bval:>15.1f} GB   {oval:>15.1f} GB")
    else:
        print(f"{key:<30}{bval:>15,.0f}     {oval:>15,.0f}")
print("=" * 70)

# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# MFU comparison
configs = ['Baseline\n(FSDP)', 'Optimized\n(5D)']
mfu_vals = [comparison['baseline']['mfu'] * 100, comparison['optimized']['mfu'] * 100]
axes[0].bar(configs, mfu_vals, color=['#FF6B6B', '#4CAF50'])
axes[0].set_ylabel('MFU (%)')
axes[0].set_title('Model FLOPs Utilization')
axes[0].axhline(y=45, color='gold', linestyle='--', label='Target (45%)')
axes[0].legend()

# Throughput comparison
tok_vals = [comparison['baseline']['tokens_per_sec'], comparison['optimized']['tokens_per_sec']]
axes[1].bar(configs, tok_vals, color=['#FF6B6B', '#4CAF50'])
axes[1].set_ylabel('Tokens/sec')
axes[1].set_title('Training Throughput')

# Memory comparison
mem_vals = [comparison['baseline']['memory_gb'], comparison['optimized']['memory_gb']]
axes[2].bar(configs, mem_vals, color=['#FF6B6B', '#4CAF50'])
axes[2].axhline(y=hw_cfg.gpu_memory_gb, color='red', linestyle='--', label='GPU limit (80 GB)')
axes[2].set_ylabel('Memory (GB)')
axes[2].set_title('Per-GPU Memory')
axes[2].legend()

plt.tight_layout()
plt.show()

**Thought Questions:**
1. By what factor did the optimized configuration improve MFU over the baseline? Is this improvement consistent with what real systems report?
2. Which single parallelism dimension contributed the most to the improvement? How would you determine this experimentally?
3. The memory reduction allows us to increase the micro-batch size. How does this affect both throughput and the effective gradient noise?

---

## 3.7 Error Analysis

In [ ]:
# TODO: Inject controlled failures and analyze the results

def analyze_failure_modes(model: ModelConfig, hw: HardwareConfig, seq_length: int):
    """
    Systematically test failure modes in parallelism configurations.

    Test the following scenarios and explain WHY each fails:

    1. TP > GPUs per node: Set TP=16 on an 8-GPU node.
       Expected: Cross-node TP has massive communication overhead.

    2. Insufficient micro-batches for PP: Set PP=8, M=1.
       Expected: 87.5% bubble fraction.

    3. Load imbalance with EP: Disable load-balancing loss.
       Expected: Expert collapse — most tokens go to 1-2 experts.

    4. Memory OOM: Set TP=1, PP=1 with long sequence.
       Expected: Activation + attention memory exceeds GPU budget.

    5. ZeRO-3 communication overhead: ZeRO-3 with TP=1 on 1024 GPUs.
       Expected: AllGather communication dominates step time.

    For each scenario:
    - Compute the relevant metrics
    - Explain the root cause of the failure
    - Suggest the fix

    Args:
        model: Model configuration
        hw: Hardware configuration
        seq_length: Sequence length

    Returns:
        List of failure analysis dictionaries, each with:
        - 'scenario': Description
        - 'config': The failing ParallelismConfig
        - 'symptom': What goes wrong (metric values)
        - 'root_cause': Why it fails
        - 'fix': How to fix it
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement failure mode analysis")


failures = analyze_failure_modes(model_cfg, hw_cfg, model_cfg.max_seq_len)
for i, f in enumerate(failures):
    print(f"\n--- Failure Mode {i+1}: {f['scenario']} ---")
    print(f"  Config: DP={f['config'].dp}, TP={f['config'].tp}, PP={f['config'].pp}, EP={f['config'].ep}")
    print(f"  Symptom: {f['symptom']}")
    print(f"  Root cause: {f['root_cause']}")
    print(f"  Fix: {f['fix']}")

**Thought Questions:**
1. Of the failure modes tested, which one would be hardest to diagnose in a real training run? Why?
2. How would you build a monitoring system to detect expert collapse during a multi-day training run?
3. Memory fragmentation is a real issue that our analytical model does not capture. How does PyTorch manage GPU memory, and why can the actual memory limit be lower than the theoretical 80 GB?

---

## 3.8 Scalability and Deployment Considerations

In [ ]:
# TODO: Scaling analysis — how does throughput scale with GPU count?

def scaling_analysis(
    model: ModelConfig,
    seq_length: int,
    gpu_counts: List[int] = [128, 256, 512, 1024, 2048, 4096],
    global_batch_size: int = 256,
) -> Dict[int, Dict[str, float]]:
    """
    Analyze how training throughput scales with GPU count.

    For each GPU count:
    1. Find the optimal parallelism configuration
    2. Record the throughput and MFU
    3. Compute the scaling efficiency: throughput(N) / (N * throughput(1_gpu))

    Args:
        model: Model configuration
        seq_length: Sequence length
        gpu_counts: List of GPU counts to evaluate
        global_batch_size: Kept constant across scales

    Returns:
        Dictionary mapping gpu_count -> performance metrics

    Hints:
        1. For each GPU count, create a HardwareConfig with appropriate nodes
        2. Call find_optimal_config for each
        3. Scaling efficiency: normalize throughput by the single-GPU compute rate
           (FLOPs / peak_tflops per GPU)
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement scaling analysis")


results = scaling_analysis(model_cfg, model_cfg.max_seq_len)

# Plot scaling curves
gpu_counts = sorted(results.keys())
throughputs = [results[n]['tokens_per_sec'] for n in gpu_counts]
mfus = [results[n]['mfu'] * 100 for n in gpu_counts]

# Ideal linear scaling
ideal_throughputs = [throughputs[0] * n / gpu_counts[0] for n in gpu_counts]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(gpu_counts, throughputs, 'o-', label='Actual', color='#4CAF50')
ax1.plot(gpu_counts, ideal_throughputs, '--', label='Ideal (linear)', color='gray')
ax1.set_xlabel('Number of GPUs')
ax1.set_ylabel('Tokens/sec')
ax1.set_title('Throughput Scaling')
ax1.legend()
ax1.set_xscale('log', base=2)
ax1.set_yscale('log', base=2)

ax2.plot(gpu_counts, mfus, 'o-', color='#2196F3')
ax2.set_xlabel('Number of GPUs')
ax2.set_ylabel('MFU (%)')
ax2.set_title('MFU vs Scale')
ax2.set_xscale('log', base=2)
ax2.axhline(y=45, color='gold', linestyle='--', label='Target (45%)')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Inference benchmarking — profile latency for different TP configurations

def inference_benchmark(
    model: ModelConfig,
    input_tokens: int = 50000,
    output_tokens: int = 2048,
    tp_degrees: List[int] = [1, 2, 4, 8],
) -> Dict[int, Dict[str, float]]:
    """
    Estimate inference latency for different Tensor Parallelism degrees.

    For inference, we care about two metrics:
    - Time to First Token (TTFT): Time to process the input (prefill)
    - Inter-Token Latency (ITL): Time per generated output token (decode)

    Args:
        model: Model configuration
        input_tokens: Number of input tokens (prefill)
        output_tokens: Number of output tokens (decode)
        tp_degrees: TP degrees to benchmark

    Returns:
        Dictionary mapping tp_degree -> latency metrics

    Hints:
        1. Prefill FLOPs ~ 2 * active_params * input_tokens
           (No factor of 3 since no backward pass)
        2. TTFT = prefill_flops / (tp_degree * peak_tflops * 1e12 * inference_efficiency)
           Use inference_efficiency ~ 0.5 (lower than training due to memory-bound ops)
        3. Decode FLOPs per token ~ 2 * active_params
           (Each new token attends to all previous tokens, but the compute is dominated
            by the linear projections)
        4. ITL = decode_flops / (tp_degree * peak_tflops * 1e12 * decode_efficiency)
           Use decode_efficiency ~ 0.15 (very memory-bound, limited by KV cache reads)
        5. Total latency = TTFT + output_tokens * ITL
    """
    # YOUR CODE HERE
    raise NotImplementedError("Implement inference benchmarking")


inf_results = inference_benchmark(model_cfg)
print(f"\nInference Benchmark ({50000} input tokens, {2048} output tokens):")
print(f"{'TP':<5}{'TTFT (ms)':<15}{'ITL (ms)':<15}{'Total (ms)':<15}")
print('-' * 50)
for tp in sorted(inf_results.keys()):
    r = inf_results[tp]
    print(f"{tp:<5}{r['ttft_ms']:<15.0f}{r['itl_ms']:<15.1f}{r['total_ms']:<15.0f}")

**Thought Questions:**
1. At what GPU count does scaling efficiency drop below 80%? What is the main bottleneck (compute, communication, or bubbles)?
2. For inference, why is the decode phase much less compute-efficient than prefill? (Hint: think about the arithmetic intensity — FLOPs per byte of memory accessed.)
3. How does the KV-cache memory scale with sequence length? At what sequence length does KV-cache memory become the bottleneck for inference?

---

## 3.9 Ethical and Regulatory Analysis

Financial AI systems carry unique ethical and regulatory responsibilities. In this section, you will assess the risks and safeguards for MeridianFin-72B.

In [ ]:
# TODO: Write an ethical impact assessment

def ethical_impact_assessment() -> str:
    """
    Write a ~500-word ethical impact assessment for MeridianFin-72B.

    Cover the following topics:

    1. DATA BIAS:
       - What biases exist in SEC filing data? (temporal, sector, size, geographic)
       - How might these biases affect model outputs?
       - What mitigation strategies would you recommend?

    2. FAIRNESS ACROSS MARKET SECTORS:
       - If the model performs better on tech company filings than energy company
         filings, what are the implications for Meridian's clients?
       - How would you measure and report sector-specific performance?
       - What is the minimum acceptable performance gap between sectors?

    3. REGULATORY COMPLIANCE:
       - What regulations govern the use of AI in financial services?
         (SEC, FINRA, MiFID II, SR 11-7)
       - How should the model's outputs be disclosed to end users?
       - What are the risks of the model producing outputs that could be
         construed as investment advice?

    4. SYSTEMIC RISK:
       - If multiple financial institutions use the same model for trading
         signals, what are the systemic risks?
       - How could model failures cascade through financial markets?

    Returns:
        A string containing the ~500-word assessment.
    """
    # YOUR CODE HERE
    raise NotImplementedError("Write the ethical impact assessment")


assessment = ethical_impact_assessment()
print(assessment)

**Thought Questions:**
1. If MeridianFin-72B is used to extract trading signals, and the model makes an error that leads to financial losses, who is liable — Meridian AI or the client? How should this be addressed contractually?
2. How would you ensure that the model does not memorize and leak confidential information from one client's documents when serving another client?
3. What is the environmental impact of training a 72B model on 1,024 H100 GPUs for 30 days? How much CO2 does this produce, and how does it compare to everyday activities?

---

## Summary

In this notebook, you have worked through the complete technical challenge facing Meridian AI:

1. **Profiled** the memory and compute requirements of training a large MoE model
2. **Established a baseline** using pure Data Parallelism (FSDP) and identified its limitations
3. **Implemented** each of the five parallelism dimensions:
   - Tensor Parallelism (column/row-parallel linear layers, Megatron MLP)
   - Pipeline Parallelism (stage partitioning, 1F1B schedule)
   - Sequence/Context Parallelism (Ring Attention)
   - Expert Parallelism (MoE with All-to-All dispatch and load balancing)
4. **Built a configuration optimizer** that searches the 5D space for the best tradeoff
5. **Evaluated** the optimized configuration against the baseline
6. **Analyzed failure modes** and scaling behavior
7. **Considered ethical and regulatory** implications

The key insight is that no single parallelism strategy is sufficient. Each dimension addresses a different bottleneck (memory, compute, sequence length, expert capacity), and the art of distributed training is finding the right combination for your specific model, hardware, and training requirements.

For further reading, see Section 4 (Production and System Design Extension) of the case study document, which covers serving infrastructure, monitoring, model versioning, and cost analysis for deploying MeridianFin-72B in production.